# Data Understanding

## 1. Load Data

In [1]:
import pandas as pd

df_customers = pd.read_csv('data/customers.csv')
df_products = pd.read_csv('data/products.csv')
df_txs = pd.read_csv('data/transactions.csv')

## 2. Data Description

In [2]:
df_customers.head()

,Customer ID,Country
0,13085.0,United Kingdom
1,13078.0,United Kingdom
2,15362.0,United Kingdom
3,18102.0,United Kingdom
4,12682.0,France


In [3]:
df_products.head()

,StockCode,Description
0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,79323P,PINK CHERRY LIGHTS
2,79323W,WHITE CHERRY LIGHTS
3,22041,"RECORD FRAME 7"" SINGLE SIZE"
4,21232,STRAWBERRY CERAMIC TRINKET BOX


In [4]:
df_txs.head()

,Invoice,InvoiceDate,Customer ID,StockCode,Quantity,Price
0,489434,2009-12-01 07:45:00,13085.0,85048,12,6.95
1,489434,2009-12-01 07:45:00,13085.0,79323P,12,6.75
2,489434,2009-12-01 07:45:00,13085.0,79323W,12,6.75
3,489434,2009-12-01 07:45:00,13085.0,22041,48,2.10
4,489434,2009-12-01 07:45:00,13085.0,21232,24,1.25


## 3. Merge Data

In [5]:
merged_df = pd.merge(df_txs, df_customers, on='Customer ID')
merged_df = pd.merge(merged_df, df_products, on='StockCode')
merged_df.head()

,Invoice,InvoiceDate,Customer ID,StockCode,Quantity,Price,Country,Description
0,489434,2009-12-01 07:45:00,13085.0,85048,12,6.95,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,580358,2011-12-02 16:07:00,17238.0,85048,3,7.95,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS
2,492761,2009-12-18 14:22:00,14911.0,85048,8,7.95,EIRE,15CM CHRISTMAS GLASS BALL 20 LIGHTS
3,507478,2010-05-10 11:32:00,14911.0,85048,6,7.95,EIRE,15CM CHRISTMAS GLASS BALL 20 LIGHTS
4,521692,2010-09-08 08:18:00,14911.0,85048,2,7.95,EIRE,15CM CHRISTMAS GLASS BALL 20 LIGHTS


In [6]:
merged_df.dtypes

Invoice          int64
InvoiceDate     object
Customer ID    float64
StockCode       object
Quantity         int64
Price          float64
Country         object
Description     object
dtype: object

# Soal 1
## Buatlah Table List Top 5 Customers yang Memiliki Nilai Transaksi Paling Banyak

In [7]:
df_top5_customers = merged_df.groupby('Customer ID').agg({'Price': 'sum'}).sort_values(by='Price', ascending=False).rename(columns={'Price': 'Total Amount'}).head(5)
df_top5_customers

,Total Amount
Customer ID,
14911.0,67977.63
17841.0,43072.84
14096.0,39324.82
14156.0,29564.44
12748.0,24413.08


# Soal 2

## Buatlah list Top 5 Produk yang memiliki nilai transaksi paling banyak

In [8]:
df_top5_products = merged_df.groupby('StockCode').agg({
    'Description': 'first', 
    'Price': 'first', 
    'Quantity': 'sum'
}).rename(columns={
    'Description': 'Product Name', 
    'Price': 'Price', 
    'Quantity': 'Total Quantity'
})

df_top5_products['Total Amount'] = df_top5_products['Price'] * df_top5_products['Total Quantity']
df_top5_products = df_top5_products.drop(columns=['Price', 'Total Quantity'])

df_top5_products = df_top5_products.sort_values(by='Total Amount', ascending=False).head(5)

df_top5_products

,Product Name,Total Amount
StockCode,,
M,Manual,8004380.80
85099B,JUMBO BAG RED WHITE SPOTTY,502180.92
85123A,WHITE HANGING HEART T-LIGHT HOLDER,349782.40
22423,REGENCY CAKESTAND 3 TIER,308346.00
23843,"PAPER CRAFT , LITTLE BIRDIE",168469.60


# Soal 3

## Buatlah table list jumlah nilai transaksi setiap bulannya dalam satu tahun

In [9]:
year = pd.DatetimeIndex(merged_df['InvoiceDate']).year.unique()
year.tolist()

[2009, 2011, 2010]

In [10]:
df_txs_2010 = merged_df[pd.DatetimeIndex(merged_df['InvoiceDate']).year == 2010].copy()
df_txs_2010.loc[:, 'Month'] = pd.DatetimeIndex(df_txs_2010['InvoiceDate']).month
df_txs_2010.loc[:, 'Total Amount'] = df_txs_2010['Quantity'] * df_txs_2010['Price']

In [11]:
df_txs_2010_sum = df_txs_2010.groupby('Month').agg({'Total Amount': 'sum'})
df_txs_2010_sum

,Total Amount
Month,
1,767614.532
2,703858.286
3,911747.431
4,790585.032
5,803094.530
6,850430.210
7,807580.890
8,819333.530
9,1092663.101


# Soal 4

## Buatlah deskripsi statistik untuk masing - masing bulan pada tahun 2010

In [12]:
df_txs_2010_desc = df_txs_2010.groupby('Month').agg({'Total Amount': 'describe'})
df_txs_2010_desc

Total Amount                                                           
             count       mean        std    min   25%    50%    75%       max
Month                                                                        
1          28238.0  27.183743  94.926656  0.001  5.90  13.20  19.90   8985.60
2          30284.0  23.241919  62.470283  0.001  5.10  12.60  19.50   3630.80
3          40774.0  22.361000  93.757848  0.001  5.04  12.60  19.50  10953.50
4          34668.0  22.804460  46.631991  0.001  6.25  13.52  19.80   1576.80
5          36672.0  21.899393  54.192250  0.100  5.80  13.20  19.80   2500.00
6          40176.0  21.167618  68.300413  0.060  5.04  12.50  19.50   8925.00
7          34852.0  23.171723  75.638946  0.120  5.80  13.20  19.80   3610.50
8          33720.0  24.298147  69.932589  0.140  5.20  13.20  19.80   2243.22
9          44515.0  24.545953  93.083844  0.001  5.90  14.75  20.16  10468.80
10         62915.0  21.344542  68.788495  0.190  4.95  12.60  19.50   5179.20
11         75683.0  20.015782  79.248305  0.001  3.90  10.20  17.70  15818.40
12         32958.0  22.491479  74.334707  0.140  4.25  11.80  19.80   3828.00

# Soal 5

## Cari Waktu Paling Sibuk dan Paling Sepi pada tahun 2010

In [13]:
df_txs_2010_copy = df_txs_2010.copy()
df_txs_2010_copy.loc[:, 'Date'] = pd.DatetimeIndex(df_txs_2010_copy['InvoiceDate']).date
df_txs_2010_copy.loc[:, 'Day'] = pd.DatetimeIndex(df_txs_2010_copy['InvoiceDate']).day_name()
df_txs_2010_copy.loc[:, 'Hour'] = pd.DatetimeIndex(df_txs_2010_copy['InvoiceDate']).hour
df_txs_2010_copy.head()

,Invoice,InvoiceDate,Customer ID,StockCode,Quantity,Price,Country,Description,Month,Total Amount,Date,Day,Hour
3,507478,2010-05-10 11:32:00,14911.0,85048,6,7.95,EIRE,15CM CHRISTMAS GLASS BALL 20 LIGHTS,5,47.7,2010-05-10,Monday,11
4,521692,2010-09-08 08:18:00,14911.0,85048,2,7.95,EIRE,15CM CHRISTMAS GLASS BALL 20 LIGHTS,9,15.9,2010-09-08,Wednesday,8
5,523302,2010-09-21 12:43:00,14911.0,85048,2,7.95,EIRE,15CM CHRISTMAS GLASS BALL 20 LIGHTS,9,15.9,2010-09-21,Tuesday,12
6,524299,2010-09-28 13:13:00,14911.0,85048,12,6.95,EIRE,15CM CHRISTMAS GLASS BALL 20 LIGHTS,9,83.4,2010-09-28,Tuesday,13
7,525013,2010-10-03 11:15:00,14911.0,85048,6,7.95,EIRE,15CM CHRISTMAS GLASS BALL 20 LIGHTS,10,47.7,2010-10-03,Sunday,11


In [14]:
df_txs_2010_date_day_hour = df_txs_2010_copy.groupby(['Date', 'Day', 'Hour']).agg({'Invoice': 'count'}).reset_index()
df_txs_2010_date_day_hour.rename(columns={'Invoice': 'Total Transactions'}, inplace=True)
df_txs_2010_date_day_hour.head()

,Date,Day,Hour,Total Transactions
0,2010-01-04,Monday,9,2
1,2010-01-04,Monday,10,29
2,2010-01-04,Monday,11,39
3,2010-01-04,Monday,12,187
4,2010-01-04,Monday,13,216


In [15]:
busiest_time = df_txs_2010_date_day_hour.loc[df_txs_2010_date_day_hour['Total Transactions'].idxmax()]
quietest_time = df_txs_2010_date_day_hour.loc[df_txs_2010_date_day_hour['Total Transactions'].idxmin()]

results = pd.DataFrame({
    'Type': ['Busiest Time', 'Quietest Time'],
    'Date': [busiest_time['Date'], quietest_time['Date']],
    'Day': [busiest_time['Day'], quietest_time['Day']],
    'Hour': [busiest_time['Hour'], quietest_time['Hour']],
    'Total Transactions': [busiest_time['Total Transactions'], quietest_time['Total Transactions']]
})

results

,Type,Date,Day,Hour,Total Transactions
0,Busiest Time,2010-06-06,Sunday,12,1070
1,Quietest Time,2010-01-19,Tuesday,16,1
